In [17]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append("../") 
from maths import baseball_stats as b


In [27]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_batting = pd.read_csv('../datafiles/Batting.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
batting = people.merge(import_batting, on='playerID', how ='inner')


In [28]:
season_batting = batting.copy()
season_batting = season_batting.drop(columns=['stint', 'teamID', 'lgID', 'birthYear', 'birthMonth', 'birthDay', 'G_old', 'G_batting']).groupby(['playerID', 'birthdate', 'yearID']).sum().reset_index()
#estimate the start of the season at april 1
season_batting['season_start'] = season_batting.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
season_batting['season_start'] = pd.to_datetime(season_batting['season_start'], errors='coerce')

#calculate the age of the player at the start of the season
season_batting['age'] = season_batting.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)

season_batting

,playerID,birthdate,yearID,G,AB,R,H,2B,3B,HR,...,CS,BB,SO,IBB,HBP,SH,SF,GIDP,season_start,age
0,aardsda01,1981-12-27,2004,11,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2004-04-01,22
1,aardsda01,1981-12-27,2006,45,2,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,2006-04-01,24
2,aardsda01,1981-12-27,2007,25,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2007-04-01,25
3,aardsda01,1981-12-27,2008,47,1,0,0,0,0,0,...,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,2008-04-01,26
4,aardsda01,1981-12-27,2009,73,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2009-04-01,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105145,zwilldu01,1888-11-02,1915,150,548,65,157,32,7,13,...,0.0,67,65.0,0.0,2.0,18.0,0.0,0.0,1915-04-01,26
105146,zwilldu01,1888-11-02,1916,35,53,4,6,1,0,1,...,0.0,4,6.0,0.0,0.0,2.0,0.0,0.0,1916-04-01,27
105147,zychto01,1990-08-07,2015,13,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015-04-01,24
105148,zychto01,1990-08-07,2016,12,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2016-04-01,25
